<a href="https://colab.research.google.com/github/isimorfizam/cake/blob/main/2_fraud_detection_exploratory_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FRAUD DETECTION - Exploratoty analysis


# 1. Introduction
♡

We are presented with a dataset of 3016 rows and 50 columns. The rows represent workspaces and their respective attributes, including, but not limited to : workspace_id - identification feature, continent of workspace owner, number of total, activated and deactivated workspace users respectively and a long list of features which quantify Clockify app usage. For a comprehensive list of features, see Section 3. Feature list.

The goal of this notebook is to explore the data at hand and try to understand which of the many features available have the greatest impact on the classification. We will use a few different approaches, all of which require a short form of preprocessing, which is done in Section 3.


The approaches for establishing feature relevance are listed in Section 4, 5 and 6 and are following :

Section 4 - Correlation analysis : Spearman's correlation coefficient used;
Section 5 - Decision Tree and Feature importance. We used 2 types of metrics : entropy and Gini index;
Section 6 - Information gain;

---

# SQL Preprocessing - Important encodings

Categorical values had already been encoded in SQL. List of categorical values and their encoding patterns goes like this :    
1. Country : represented by its continent, then continent encoded by its frequency of appearance. Found in column 'freq'. Frequency encoding goes like this :
'America' : 59
'Europe' : 26
'Asia' : 9
'Australia' : 4
'Africa' : 2
2. Email : 1 if gmail.com, yahoo.com, outlook.com or yandex.com, ELSE 0; found in email_enc
3. Account type : 0 if CAKE_INC, 1 if CAKE_AG, 2 if both appeared for the same workspace_id.


---

# Short review of results

1. Spearman's coeficient showed that the most correlated features are : email_enc (+0.1) and account_enc_2 (+0.08), freq(-0.08) and project_creation(0.09).
2. In both approaches used to build a Decision Tree (Gini index and entropy), it was found that project creation greater than 0.5 means that there was no fraud.
3. With project creation lower than 0.5 and email_enc < 0.5, which is email_enc = 0, which is non-standard mail (not gmail, yahoo etc)
4. Account_enc_2 < 0.5, which is 0, meaning CAKE_INC account, suggests 'no fraud'
5. Information gains between features and target variable suggest most important features are : 'project_creation','max_plan','max_logins', 'freq', 'email_enc', which is supported by other analysis as well.
6. Features that are decided as irrelevant based on the dataset we have, and that could be dropped from future usage are following :

'gps_tracking',
 'time_entries_timesheet',
 'trial_activation',
 'screenshot_capturing',
 'duration_format',
 'activate_scheduling',
 'publish_schedule',
 'connect_calendar',
 'deactivated_users',
 'invited_users',
 'tracked_time_timesheet',
 'connect_quickbooks',
 'project_budget',
 'timesheet_approvals',
 'add_role',
 'activate_audit_log',
 'add_targets',
 'create_expense',
 'integrations',
 'time_entries_calendar',
 'max_plan',
 'max_share_report',
 'create_invoice',
 'create_custom_field',
 'tracked_time_calendar',
 'add_delete_tag',
 'lock_entries',
 'rounding_reports',
 'time_off',
 'import_csv'.  These were found as an intersection of features with information gain less than 0.02 and correlation less than 0.02

 7. Boruta algorythm's green zone features :
  ['activated_users',
  'total_users',
  'project_creation',
  'add_client',
  'export_report',
  'time_entries_tracker',
  'tracked_time_tracker',
  'add_new_member',
  'time_entries_calendar',
  'time_entries_timesheet',
  'tracked_time_timesheet',
  'add_filter',
  'account_enc_2',
  'max_logins',
  'email_enc',
  'days_to_purchase'] and blue zone features :['create_invoice', 'tracked_time_calendar', 'freq']

8. Some contradictory results between Boruta and other approaches. Boruta gives more importance to following :    

 ['time_entries_calendar', 'tracked_time_timesheet', 'add_new_member', 'time_entries_timesheet']
['create_invoice', 'tracked_time_calendar']



# Installation and data loading

In [1]:
# @title
!pip install --quiet ydata_profiling
!pip install --quiet pycaret
!pip install --quiet tqdm
!pip install --quiet numpy==1.19.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# @title
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# open Google Sheets document as CSV
import pandas as pd
pd.set_option('display.max_columns',None,'display.max_rows',None)

url = 'https://docs.google.com/spreadsheets/d/1TqnAXmB_9JM5c4MRewyE438NculxqsZrnOcQ2Rb-iW0/export?format=csv'
df = pd.read_csv(url)



# 2. Example of data


In [ ]:
# Making sure the frame is loaded correctly, getting a preview of the data
df.head()

Check for duplicates

In [ ]:
# @title
print(df.duplicated().sum())
# df.drop_duplicates(inplace=True)
# df.reset_index(inplace=True)

# 3. Feature list and preprocessing

In this section we deal with missing values, list the features and try to get some understanding of how the features are distributed over the dataset.


In the first paragraph we are presented with a list of all features, their data types, and a number of non-null values in each feature, which we use to find and understand missing values.


In [ ]:
# @title
# Finding rows with missing values and checking data types
df.info()

Next, we check the number of unique values for each column and drop columns with only one value, since those bring no relevant information.
In this case, we only have one column with a single value across the dataset - the 'duplucates' column. This column was artificially added and it is a bool value : 'True' if the same workspace apeared more that once in any of the tables used for creating the dataset and 'False' otherwise. It is expected for each workspace to have more apearances, ergo, for 'duplicates' feature to always be truthful, since we used tables that mark each action in the workspace.

In [ ]:
# Checking number of unique values for each column / If ever there was 1, for example, we would drop that column
df.nunique()

In [ ]:
df.drop(columns=['duplicates','cnt'],inplace=True);

Now, we deal with columns that have null values, aka missing values. Those include : 'tracked_time_calendar', 'tracked_time_timesheet' and 'tracked_time_tracker'. Since these columns are measures of certain events, their activation is triggered when an event happens (when time tracking is done through one of these options). Hence, missing values represent absence of the event, which is equivalent to a duration of 0. And that is how we will represent null.

In [ ]:
df['tracked_time_tracker'].fillna(0,inplace=True)
df['tracked_time_calendar'].fillna(0,inplace=True)
df['tracked_time_timesheet'].fillna(0,inplace=True)

df['days_to_purchase'].fillna(-1,inplace=True)

In [ ]:
summary_statistics = df.describe(percentiles=(0.25,0.50,0.75,1)).round(3)

In [ ]:
summary_statistics


In [ ]:
# Checking distributions.
# if Xmean = Xmedian (2nd Quartile/50%) then it is normal distribution (Gaussian).
# if Xmean > Xmedian (2nd Quartile/50%) then it is right-skew/positive-skew distribution.
# if Xmean < Xmedian (2nd Quartile/50%) then it is left-skew/negative-skew distribution.

for column in summary_statistics:
  if summary_statistics[column].loc['mean'] == summary_statistics[column].loc["50%"]:
    if summary_statistics[column].loc['min'] < 0:
      print(f"{column} data is normally distributed and has negative values.")
    else:
      print(f"{column} data is normally distributed and doesn't have negative values.")
  elif summary_statistics[column].loc['mean'] > summary_statistics[column].loc["50%"]:
    if summary_statistics[column].loc['min'] < 0:
      print(f"{column} data is positive-skew/right-skew distributed and has negative values.")
    else:
      print(f"{column} data is positive-skew/right-skew distributed and doesn't have negative values.")
  else:
    if summary_statistics[column].loc['min'] < 0:
      print(f"{column} data is negative-skew/left-skew distributed and has negative values.")
    else:
      print(f"{column} data is negative-skew/left-skew distributed and doesn't have negative values.")

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
# standardization
X = df.drop(columns=['fraud','workspace_id','continent','creation_date'])
y = df['fraud']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# X_train_std = (X_train - X_train.mean())/(X_train.std())
# X_test_std = (X_test - X_train.mean())/(X_train.std())

# X_train_std.dropna(axis=1, inplace=True)
# X_test_std.dropna(axis=1, inplace=True)

# 4.Correlation analisys

In this section we use Spearmans correlation coefficient to calculate the correlation of each feature with the target variable.

In [ ]:
correlations = {}
p_values = {}

def correlation_analysis0(data,target,coefficient):
  for column in data.drop(target,axis=1):
    r,p = coefficient(data[column],data[target])
    correlations[column]=r
    p_values[column]=p

def correlation_analysis(X : pd.DataFrame, y : pd.Series, coefficient)  -> pd.DataFrame :
  corr = {}
  df_r = pd.DataFrame(columns = X.columns)
  df_p = pd.DataFrame(columns = X.columns)
  for column in X.columns:
    r, p = coefficient(X[column],y)
    df_r[column] = pd.Series(r)
    df_p[column] = pd.Series(p)

  df = pd.concat([df_r,df_p],axis=0)
  df.index = ['corr','p_value']
  return df

In [ ]:
# Checking correlations.
# If your data is normally distributed, use Pearson Product-Moment Correlation.
# If your data is skewed and you have monotonic relationship, use rank coefficients: Spearman for larger and Kendall's Tau for smaller datasets.
from scipy.stats import pearsonr,spearmanr,kendalltau

corr = correlation_analysis(X,y,coefficient = spearmanr)
corr.round(2)
corr.sort_values(by='corr',axis=1,ascending=True)

In [ ]:
corr.sort_values(by='p_value',axis=1,ascending=False)

#  5. Decision Tree and Feature importance


In [ ]:
#Checking the nodes structure.

dec_tree_ = DecisionTreeClassifier(max_depth=4, criterion="entropy",random_state=42).fit(X,y)

fig,ax = plt.subplots(figsize=(15,10))
plot_tree(dec_tree_, feature_names=X.columns, class_names=["not-fraud","fraud"])
fig.savefig("dec_tree.png")

In [ ]:
#Checking the nodes structure.

dec_tree_ = DecisionTreeClassifier(max_depth=4, criterion="gini",random_state=42).fit(X,y)

fig,ax = plt.subplots(figsize=(15,10))
plot_tree(dec_tree_, feature_names=X.columns, class_names=["not-fraud","fraud"])
fig.savefig("dec_tree.png")

# 5. Information gain - feature based

In [ ]:
from sklearn import tree
print(tree.export_text(dec_tree_))

In [ ]:
import numpy as np
import math

def calc_entropy(x):
    counts = np.bincount(x) # number of each unique value in a column
    probability = counts/(len(x))
    entropy = 0
    for prob in probability:
        if prob >0:
            entropy += prob * math.log(prob, 2)
    return -entropy

def information_gain(X, column, y):

    original_entropy = calc_entropy(y)
    values = X[column].unique()

    X_0 = X[X[column] == values[0]]
    X_1 = X[X[column] == values[1]]

    y_0 = y[X[column]==values[0]]
    y_1 = y[X[column]==values[1]]

    prob_0 = (X_0.shape[0])/X.shape[0]
    prob_1 = (X_1.shape[0])/X.shape[0]

    return  original_entropy - prob_0 * calc_entropy(y_0) - prob_1*calc_entropy(y_1)

In [ ]:
gains_cols = dict.fromkeys(X.columns)
for column in X.columns :
  gains_cols[column] = (information_gain(X,column, y))
  #print('Information gain for feature ' + column + ' equals ' + str(information_gain(X,column, y)))

temp = pd.DataFrame(sorted(gains_cols.items(), key=lambda item: item[1]),columns=['feature','information_gain'])

non_important5 = temp['feature'][temp['information_gain']<0.02]

In [ ]:
temp.sort_values(by ='information_gain')

# 6. Random Forest and Feature Importance

In [ ]:
#Training RandomForestClassifier model.
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight="balanced",max_depth=3,criterion="gini").fit(X,y)

In [ ]:
feature_importances = rfc.feature_importances_
importance_df_rf = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
importance_df_rf = importance_df_rf.sort_values(by='Importance', ascending=False)
importance_df_rf['Cumulative Importance'] = importance_df_rf['Importance'].cumsum()
important_features_df = importance_df_rf[importance_df_rf['Importance'] > 0.02]
figure2 = plt.figure(figsize=(6, 4))
plt.barh(important_features_df['Feature'], important_features_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.savefig("feature_importance.png")
plt.show()

In [ ]:
non_important6 = importance_df_rf[importance_df_rf['Importance'] < 0.02].Feature.values

non_important6

In [ ]:
non_important = list(set(list(non_important5.values)).intersection(list(non_important6)))
non_important

In [ ]:
# from ydata_profiling import ProfileReport

# profile = ProfileReport(df, title="Profiling Report")
# profile.to_notebook_iframe()
# profile.to_file("fraud_detection_exploratory_data_analysis.html")

# 7. Boruta Algorythm

Boruta Algorythm is a feature selection algorythm that bases its decision making on the assumption that feature importance is a


The importance of a feature of a single decision tree is calculated as the difference in performance between the model using the original features versus the model using the permuted features divided by the number of examples in the training set. The importance of a feature is the average of the measurements across all trees for that feature.

Steps :
1. Create a copy of the training set features and merges them with the original features. These synthetic features are called shadow features.
2. Shuffle, aka, create randomized permutations of these shadow features. This is done at each iteration.
3. Train RF and compute the z-score of all original and synthetic features.
4. Find the maximum maximum importance of all synthetic features. A non-synthetic feature will be considered relevant if its importance is greater than this maximum.
5. Repeat steps 2,3,4 for a number of iterations and each time take note of which original features made the cut.
6. Use binomial distribution to finalize which features provide enough confidence to be kept in the final list.

In [ ]:
# from warnings import simplefilter
# simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# # steps 1 and 2
# X_new = X.copy()
# for column in X.columns:
#     X_new[f"shadow_{column}"] = X_new[column].sample(frac=1).reset_index(drop=True)

# def boruta_manual(X : pd.DataFrame, y : pd.Series) -> list:
#   # steps 3,4,5
#   rfc = RandomForestClassifier(class_weight="balanced",max_depth=4,criterion="gini").fit(X_new,y)
#   importances = {feature_name: f_importance for feature_name, f_importance in zip(X_new.columns, rfc.feature_importances_)}
#   only_shadow_feat_importance = {key:value for key,value in importances.items() if "shadow" in key}
#   highest_shadow_feature = list(dict(sorted(only_shadow_feat_importance.items(), key=lambda item: item[1], reverse=True)).values())[0]
#   selected_features = pd.DataFrame([(key, value) for key, value in importances.items() if value > highest_shadow_feature], columns = ['Feature', 'Importance'])
#   return selected_features


# from tqdm import tqdm
# TRIALS = 50
# feature_hits = {i:0 for i in X.columns}
# for _ in tqdm(range(TRIALS)):
#     imp_features = boruta_manual(X, y)

#     for key, _ in feature_hits.items():
#         if key in imp_features['Feature'].values :
#            feature_hits[key] += 1

# feature_hits

# # Calculate the probability mass function
# # A binomial distribution with a probability 0.5 has a bell-shaped curve with 5% of the overall probability in the tails.
# import scipy
# pmf = [scipy.stats.binom.pmf(x, TRIALS, .5) for x in range(TRIALS + 1)]
# #pmf

# def get_tail_items(pmf):
#   total = 0
#   for i, x in enumerate(pmf):
#     total += x
#     if total >= 0.05:
#       break
#   return i

# plt.plot([i for i in range(TRIALS + 1)], pmf,"-o")
# plt.title(f"Binomial distribution for {TRIALS} trials")
# plt.xlabel("No. of trials")
# plt.ylabel("Probability")
# plt.grid(True)


# # select features from n number of trials
# def choose_features(feature_hits, TRIALS, thresh):
#   #define boundries
#   green_zone_thresh = TRIALS - thresh
#   blue_zone_upper = green_zone_thresh
#   blue_zone_lower = thresh

#   green_zone = [key for key, value in feature_hits.items() if value >= green_zone_thresh]
#   blue_zone = [key for key, value in feature_hits.items() if (value >= blue_zone_lower and value < blue_zone_upper)]

#   return green_zone, blue_zone

# thresh = get_tail_items(pmf)
# green, blue = choose_features(feature_hits, TRIALS, thresh)

In [ ]:
!pip install --quiet  boruta


In [ ]:

from boruta import boruta_py

with open('boruta_py.txt','r') as writer_file:
    contents_to_write = writer_file.read()
with open(boruta_py.__file__,'w') as file_to_overwrite:
    file_to_overwrite.write(contents_to_write)
rfc = RandomForestClassifier(class_weight="balanced",max_depth=4,criterion="gini").fit(X,y)


from boruta import BorutaPy
# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=rfc,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

# train Boruta
# N.B.: X and y must be numpy arrays
x_arr = np.array(X)
y_arr = np.array(y)
feat_selector.fit(x_arr, y_arr)

# # print support and ranking for each feature
# print("\n------Support and Ranking for each feature------")
# for i in range(len(feat_selector.support_)):
#     if feat_selector.support_[i]:
#         print("Passes the test: ", X.columns[i],
#               " - Ranking: ", feat_selector.ranking_[i])
#     else:
#         print("Doesn't pass the test: ",
#               X.columns[i], " - Ranking: ", feat_selector.ranking_[i])



number_important = np.sum(feat_selector.support_ + feat_selector.support_weak_)
all_features = len(X.columns)
very_important = sum(feat_selector.support_)

print(f'Number of features:{all_features}')
print(f'Number of important features:{number_important} - out of which {very_important} very important features.')
print(pd.Series(feat_selector.ranking_[feat_selector.support_], index=X.columns[feat_selector.support_], name='Boruta Analysis - Supported'))
print(pd.Series(feat_selector.ranking_[feat_selector.support_weak_], index=X.columns[feat_selector.support_weak_], name='Boruta Analysis - Weak'))
print(pd.Series(feat_selector.ranking_[1 - (feat_selector.support_weak_+feat_selector.support_)], index=X.columns[1 - (feat_selector.support_weak_+feat_selector.support_)], name='Boruta Analysis - Not supported'))



# PyCaret

In [ ]:
from pycaret.classification import *
import seaborn as sns
sns.set(color_codes=True)

# Checking the models: hyperparameters.

s = setup(data = df, target='fraud', session_id=42)

best = compare_models()